# Data Preprocessing Fires 2 #
in this notebook we aim to predict the statecode of a fire occurance based on the lat/long values we have.
We ended up using the KNN Algorithm as it proved to have the highest accuracy compared to other common classification algorighms.

To train the model we used the ~50.000 accurate values we got from the google geocoding API in the last notebook.

In [ ]:
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
drive.mount("/content/drive")

Mounted at /content/drive


- Loading the rand_fires (the 50.000 values from the API were randomized in advance) as our training/testing dataset.
- Loading the rand_fires_empty which includes the rest of the randomized fires without statecode into df_total.

In [ ]:
df = pd.DataFrame()
df = df.append(pd.read_csv(f'/content/drive/Shared drives/BNCS411_Final_Project/rand_fires.csv', delimiter=","))
df_total = pd.DataFrame()
df_total = df_total.append(pd.read_csv(f'/content/drive/Shared drives/BNCS411_Final_Project/rand_fires_empty.csv', delimiter=","))
#df_total = df_total.append(pd.read_csv(f'/content/drive/Shared drives/BNCS411_Final_Project/fires_pred_knn.csv', delimiter=","))

- Unnamed: 0 has to be dropped as we don't need it and still haven't found out how to avoid creating it.
- Extracting a version of df_total which only contains lat/long as these are the labels used by knn (we don't want any other columns to be present when predicting)

In [ ]:
del df['Unnamed: 0']
del df_total['Unnamed: 0']
df_total = df_total[df_total['statecode'] != 'BR']
df_total_no_date = df_total.drop(['acq_date', 'statecode'], 1)
df_total_no_date

,latitude,longitude
0,-2.9955,-41.3716
1,-9.4152,-51.4789
2,-11.9844,-46.6188
3,-11.0440,-47.3342
4,-3.3384,-51.4488
...,...,...
7252682,-9.4897,-65.7249
7252683,-11.6497,-61.4685
7252684,-11.3062,-57.9672
7252685,-9.8674,-62.4461


- Preparing our labels & features on the training/testing set

In [ ]:
X = df.drop(['acq_date', 'statecode'], 1)
y = df['statecode']
#X, y

- Translating lat/long to radians to use the haversine metric later on

In [ ]:
X_rad = X*np.pi/180
y_rad = y

- Creating Train/Test Splits (tune here to improve accuracy)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_rad, y_rad, test_size = 0.1, random_state = 42)
#X_train, X_test, y_train, y_test

- Fitting the model to our training data

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5, metric='haversine').fit(X_train, y_train)
accuracy = knn.score(X_test, y_test)
accuracy

0.8548748219010788

- change lat/long of df_total to radians
- save the prediction onto 'statecode' column of our df_total

In [ ]:
df_total_no_date_rad = df_total_no_date*np.pi/180
df_total['statecode'] = knn.predict(df_total_no_date_rad)

- Finally drop lat/long as we don't need it anymore after predicting the Statecode

In [ ]:
df_total = df_total.drop(['latitude', 'longitude'], 1)

- group to count Fires per day/state and save it onto 'fires column' 

In [ ]:
df_total = df_total.groupby(["acq_date", "statecode"]).size().reset_index(name="fires")
df_total

- saving result to csv file

In [ ]:
df_total.to_csv('fires_pred_knn.csv')
!cp fires_pred_knn.csv "/content/drive/Shared drives/BNCS411_Final_Project"

- previous accuracy tests to determine optimal algorithm for our dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)
X_train, X_test, y_train, y_test
dtc = DecisionTreeClassifier(max_depth=100, criterion='entropy', min_samples_split=2).fit(X_train, y_train)
accuracy = dtc.score(X_test, y_test)
accuracy

**Preprocessing TODO**
- ~~combine Weather & fire dataset~~
- maybe combine entire the preprocessing code into single file